In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet("../output/validated_dataset.parquet")

In [3]:
df.head()

,db_path,question,query,answer,is_valid,error
0,/Users/michaelfrantz/dev/CalTech-CTME-AramCo-2...,How many orders were placed in the most recent...,SELECT COUNT(*) FROM orders WHERE order_date >...,[[23]],True,None
1,/Users/michaelfrantz/dev/CalTech-CTME-AramCo-2...,What is the total revenue generated from order...,SELECT SUM(total_amount) FROM orders WHERE ord...,[[14745.789999999999]],True,None
2,/Users/michaelfrantz/dev/CalTech-CTME-AramCo-2...,Who are the top 5 customers by the number of o...,"SELECT c.first_name, c.last_name, COUNT(o.id) ...","[[""Tammy"", ""Torres"", 5], [""Miranda"", ""Moore"", ...",True,None
3,/Users/michaelfrantz/dev/CalTech-CTME-AramCo-2...,Which restaurant has the highest number of com...,"SELECT r.name, COUNT(o.id) as completed_orders...","[[""Jenkins PLC"", 6]]",True,None
4,/Users/michaelfrantz/dev/CalTech-CTME-AramCo-2...,What is the average order amount for each rest...,"SELECT r.name, AVG(o.total_amount) as avg_orde...","[[""Duffy, Hardin and Gill"", 621.0749999999999]...",True,None
